In [1]:
import requests, json, gzip, os
import pandas as pd
from datetime import datetime

In [2]:
with open("Dados/attacks_012023.json", "r", encoding="utf-8") as f:
    attacks = f.readlines()
f.close()
print(len(attacks))
print(attacks[0])

74283
{"_id":{"$oid":"65a01f98a2e956442d19f540"},"block_number":16308191,"block_timestamp":1672531223,"first_transaction":{"hash":"0x155fd4919a33a9b41480ceed4a3a68eaa43b6608eb6c9657a63a97fecac396e7","accessList":[{"address":"0xE0554a476A092703abdB3Ef35c80e0D76d32939F","storageKeys":[{"$binary":{"base64":"AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAI=","subType":"00"}},{"$binary":{"base64":"611nDCkdypjPji6yv6ThFJlGDSBgWcj7h3etKIKeiss=","subType":"00"}},{"$binary":{"base64":"611nDCkdypjPji6yv6ThFJlGDSBgWcj7h3etKIKeisw=","subType":"00"}},{"$binary":{"base64":"AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA=","subType":"00"}},{"$binary":{"base64":"AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAg=","subType":"00"}},{"$binary":{"base64":"AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAE=","subType":"00"}},{"$binary":{"base64":"1b9jyS6Y69kGuiYg1ppDR4Z6JvMJCcxVZ+Ez7EwJ0sY=","subType":"00"}},{"$binary":{"base64":"1b9jyS6Y69kGuiYg1ppDR4Z6JvMJCcxVZ+Ez7EwJ0sc=","subType":"00"}},{"$binary":{"base64":"1b9jyS6Y69kGui

In [20]:
url_zeromev = "https://data.zeromev.org/v1"

for attack in attacks:
    attack_formatted = json.loads(attack)

    response = requests.get(url_zeromev + "/mevBlock", params={
        "block_number": attack_formatted["block_number"],
        "count": 1
    })

    block_datetime = datetime.strptime(response.json()[0]["arrival_time_eu"].split(".")[0], "%Y-%m-%dT%H:%M:%S")
    mempool_filename = "Mempool/{}{:02d}{:02d}_{:02d}.csv.gz".format(block_datetime.year, block_datetime.month, block_datetime.day, block_datetime.hour)
    print(block_datetime)


    if not os.path.exists(mempool_filename):
        url_blocknative = "https://archive.blocknative.com/{}{:02d}{:02d}/{:02d}.csv.gz".format(block_datetime.year, block_datetime.month, block_datetime.day, block_datetime.hour)
        response = requests.get(url_blocknative, stream=True)
        if response.status_code == 200:
            f = open(mempool_filename, "wb")
            f.write(response.raw.read())
            f.close()
        else:
            print("ERRO: {}".format(response.status_code))
    
    f = gzip.open(mempool_filename)
    df_mempool = pd.read_csv(f, sep="\t")
    f.close()
    
    tx_hash_ta1 = attack_formatted["first_transaction"]["hash"]
    tx_hash_tv = attack_formatted["whale_transaction"]["hash"]
    tx_hash_ta2 = attack_formatted["second_transaction"]["hash"]
    
    print(df_mempool.loc[(df_mempool["hash"] == tx_hash_ta1) & (df_mempool["status"] == "pending")])
    print(df_mempool.loc[(df_mempool["hash"] == tx_hash_tv) & (df_mempool["status"] == "pending")])
    print(df_mempool.loc[(df_mempool["hash"] == tx_hash_ta2) & (df_mempool["status"] == "pending")])
    break

2023-01-01 00:00:28
Empty DataFrame
Columns: [detecttime, hash, status, region, reorg, replace, curblocknumber, failurereason, blockspending, timepending, nonce, gas, gasprice, value, toaddress, fromaddress, input, network, type, maxpriorityfeepergas, maxfeepergas, basefeepergas, dropreason, rejectionreason, stuck, gasused, detect_date]
Index: []

[0 rows x 27 columns]
                      detecttime  \
144687  2023-01-01T00:00:03.797Z   
144693  2023-01-01T00:00:03.820Z   
144712  2023-01-01T00:00:03.905Z   

                                                     hash   status  \
144687  0xbe2ac237061b744bd788485a10618c019c72c1ccb173...  pending   
144693  0xbe2ac237061b744bd788485a10618c019c72c1ccb173...  pending   
144712  0xbe2ac237061b744bd788485a10618c019c72c1ccb173...  pending   

                region  reorg replace  curblocknumber failurereason  \
144687       us-east-1    NaN     NaN        16308189          none   
144693    eu-central-1    NaN     NaN        16308189       

C:\Users\josue\AppData\Local\Temp\ipykernel_5476\3957241747.py:27: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  df_mempool = pd.read_csv(f, sep="\t")
